In [1]:
import spiceypy as spice
import spiceypy.utils.support_types as stypes
import pandas as pd
import numpy as np
import glob

In [75]:
def calculate_eclipses(occulted, occulting, observer, start, stop):
    path = "/home/dominik/dev/tudat-bundle/spice/lro/data"
    for file in glob.glob(f"{path}/spk/*.bsp"):
        spice.furnsh(file)
    spice.furnsh(f"{path}/lsk/naif0012.tls")
    spice.furnsh(f"{path}/pck/pck00010.tpc")
    
    TDBFMT = 'YYYY-MM-DD HR:MN:SC.### TDB ::TDB'

    stepsize = 300.0
    
    confine = stypes.SPICEDOUBLE_CELL(2)
    spice.wninsd( spice.str2et(start), spice.str2et(stop), confine)

    occultations = []
    for occ_type in ["FULL", "ANNULAR", "PARTIAL"]:
        occultation_windows = stypes.SPICEDOUBLE_CELL(10000)
        spice.gfoclt( occ_type,
                     occulting,    "ellipsoid",  f"iau_{occulting}",
                        occulted,     "ellipsoid",  f"iau_{occulted}",
                         "none", observer,  stepsize, confine, occultation_windows)

        for i in range(spice.wncard( occultation_windows )):
            [intbeg, intend] = spice.wnfetd(occultation_windows, i)
            occultations.append((spice.timout(intbeg, TDBFMT), spice.timout(intend, TDBFMT), occ_type))
    occultations.sort()
    
    print(f"{observer} observing {occulted} occultation due to {occulting}:")
    if not occultations:
        print(" - No occultations")
    for t_start, t_end, occ_type in occultations:
        print(f" - {t_start} - {t_end} ({occ_type})")

    spice.kclear()

In [76]:
calculate_eclipses("Sun", "Earth", "Moon", "2010 JUNE 26", "2010 JUNE 27")

Moon observing Sun occultation due to Earth:
 - 2010-06-26 09:36:49.510 TDB - 2010-06-26 11:28:04.861 TDB (PARTIAL)
 - 2010-06-26 11:28:04.861 TDB - 2010-06-26 11:52:27.754 TDB (FULL)
 - 2010-06-26 11:52:27.754 TDB - 2010-06-26 13:43:44.488 TDB (PARTIAL)


## Check eclipses given in Tooley 2010:

In [71]:
calculate_eclipses("Sun", "Earth", "LRO", "2009 DEC 30", "2010 JAN 1")
calculate_eclipses("Sun", "Moon", "LRO", "2009 DEC 30", "2010 JAN 1")

LRO observing Sun occultation due to Earth:
 - 2009-12-31 17:47:13.241 TDB - 2009-12-31 20:52:34.914 TDB (PARTIAL)
LRO observing Sun occultation due to Moon:
 - No occultations


In [69]:
calculate_eclipses("Sun", "Earth", "LRO", "2010 JUNE 25", "2010 JUNE 27")
calculate_eclipses("Sun", "Moon", "LRO", "2010 JUNE 25", "2010 JUNE 27")

LRO observing Sun occultation due to Earth:
 - 2010-06-26 09:53:38.797 TDB - 2010-06-26 10:23:59.617 TDB (PARTIAL)
 - 2010-06-26 10:23:59.617 TDB - 2010-06-26 10:52:58.911 TDB (FULL)
 - 2010-06-26 10:52:58.911 TDB - 2010-06-26 12:28:03.003 TDB (PARTIAL)
 - 2010-06-26 12:28:03.003 TDB - 2010-06-26 12:57:02.536 TDB (FULL)
 - 2010-06-26 12:57:02.536 TDB - 2010-06-26 13:27:40.126 TDB (PARTIAL)
LRO observing Sun occultation due to Moon:
 - No occultations


In [70]:
calculate_eclipses("Sun", "Earth", "LRO", "2010 DEC 20", "2010 DEC 22")
calculate_eclipses("Sun", "Moon", "LRO", "2010 DEC 20", "2010 DEC 22")

LRO observing Sun occultation due to Earth:
 - 2010-12-21 05:47:02.614 TDB - 2010-12-21 07:28:19.694 TDB (PARTIAL)
 - 2010-12-21 07:28:19.694 TDB - 2010-12-21 09:12:40.909 TDB (FULL)
 - 2010-12-21 09:12:40.909 TDB - 2010-12-21 10:53:32.804 TDB (PARTIAL)
LRO observing Sun occultation due to Moon:
 - No occultations


In [67]:
calculate_eclipses("Sun", "Earth", "LRO", "2011 JUN 14", "2011 JUN 16")
calculate_eclipses("Sun", "Moon", "LRO", "2011 JUN 14", "2011 JUN 16")

LRO observing Sun occultation due to Earth:
 - 2011-06-15 17:36:10.340 TDB - 2011-06-15 19:08:19.775 TDB (PARTIAL)
 - 2011-06-15 19:08:19.775 TDB - 2011-06-15 21:17:29.520 TDB (FULL)
 - 2011-06-15 21:17:29.520 TDB - 2011-06-15 22:49:39.241 TDB (PARTIAL)
LRO observing Sun occultation due to Moon:
 - No occultations
